In [14]:
import pandas as pd
import camelot

def clean(df):
    # Skipping the first few header rows which don't contain actual data
    for j in range(0, df.shape[0]):
        try:
            int(df.iloc[j, 0])
            df = df.iloc[j:].reset_index(drop=True)
            break
        except:
            continue
    return df

def append_table(tables):
    df = pd.DataFrame()
    for i in range(tables.n):
        df = pd.concat([df, clean(tables[i].df)], ignore_index=True)
    return df

def process_pdf(file_path, year):
    try:
        tables = camelot.read_pdf(file_path, pages='all', flavor='stream')
        df = append_table(tables)
        
        # Define the expected columns
        expected_columns = [
            'Sr No', 'Subsidiary', 'Country', 'Reporting currency', 'Exchange Rate', 
            'Share capital (incl. advances towards capital where applicable)', 
            'Reserves and Surplus', 'Total Assets', 'Total Liabilities', 
            'Turnover', 'Profit/(Loss) Before Tax', 'Tax Expense/(Credit)', 
            'Profit/(Loss) after tax', 'Profit/(Loss) for the period/year *', 
            'Proposed dividend and tax thereon', 'Investments (except in case of investment in the subsidiaries)', 
            '% of shareholding'
        ]
        
        # Adjust the columns of the dataframe
        if df.shape[1] >= len(expected_columns):
            df = df.iloc[:, :len(expected_columns)]  # Take only the required number of columns
            df.columns = expected_columns
        else:
            df.columns = expected_columns[:df.shape[1]]
        
        df['Year'] = year
        return df
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return pd.DataFrame()

# List of PDF files and the corresponding years
pdf_files = [
    ('Subsidiaries-2014-15.pdf', '2014-15'),
    ('Subsidiaries-2015-16.pdf', '2015-16'),
    ('Subsidiaries-2016-17.pdf', '2016-17'),
    ('Subsidiaries-2017-18.pdf', '2017-18'),
    ('Subsidiaries-2018-19.pdf', '2018-19'),
    ('Subsidiaries-2019-20.pdf', '2019-20'),
    ('Subsidiaries-2020-21.pdf', '2020-21'),
    ('Subsidiaries-2022-23.pdf', '2022-23'),
    ('Subsidiaries-2023-24.pdf', '2023-24')
]

# Initialize an empty DataFrame to store the consolidated data
consolidated_df = pd.DataFrame()

# Process each PDF file and append the data to the consolidated DataFrame
for file, year in pdf_files:
    pdf_df = process_pdf(file, year)
    consolidated_df = pd.concat([consolidated_df, pdf_df], ignore_index=True)

# Convert numerical columns to the appropriate data type
numerical_columns = [
    'Exchange Rate', 'Share capital (incl. advances towards capital where applicable)', 
    'Reserves and Surplus', 'Total Assets', 'Total Liabilities', 
    'Turnover', 'Profit/(Loss) Before Tax', 'Tax Expense/(Credit)', 
    'Profit/(Loss) after tax', 'Profit/(Loss) for the period/year *', 
    'Proposed dividend and tax thereon', 'Investments (except in case of investment in the subsidiaries)', 
    '% of shareholding'
]

for column in numerical_columns:
    consolidated_df[column] = consolidated_df[column].replace(',', '', regex=True)
    consolidated_df[column] = pd.to_numeric(consolidated_df[column], errors='coerce')

# Drop any remaining rows with NaN values
consolidated_df.dropna(how='all', inplace=True)

# Display the consolidated data
consolidated_df

# Save the consolidated DataFrame to an Excel file
consolidated_df.to_excel('Consolidated_Subsidiary_Data_Cleaned.xlsx', index=False)


                                  Sr No  \
0                                     1   
1                                     2   
2                                     3   
3                                     4   
4                                     5   
...                                 ...   
1860                                      
1861                                      
1862  (subsidiary w.e.f March 30, 2004)   
1863                   PT  Tata  Motors   
1864                 December 29, 2011)   

                                        Subsidiary      Country  \
0                  TAL Manufacturing Solutions Ltd        India   
1                               TML Drivelines Ltd        India   
2                      Concorde Motors (India) Ltd        India   
3                             Sheba Properties Ltd        India   
4           Tata Daewoo Commercial Vehicle Co. Ltd  South Korea   
...                                            ...          ...   
1860  Consolid